In [ ]:
import nest_asyncio
from llama_index.core import SimpleDirectoryReader


nest_asyncio.apply()

import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


single_document = SimpleDirectoryReader(input_files=['cases/case4.txt']).load_data()

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SummaryIndex, VectorStoreIndex

splitter = SentenceSplitter(chunk_size=8192)
nodes = splitter.get_nodes_from_documents(single_document)
index = VectorStoreIndex(nodes)
retriever = index.as_retriever()

from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Gemini(api_key=GEMINI_API_KEY)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from llama_index.core import PromptTemplate
qa_prompt = PromptTemplate(
    '''

        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"

        please go through the above case in complete detail and provide me a contextual summary of the case, A summary of the relevant legal information.\nI want large data. give large contextual summary full detailed case by going through the whole txt file. give proper detailed info of what all happened in the case arguments by appealant, responsdent, judge decisions, etc etc,\nI want everything in detail. all the facts in the case, all relevant sections in the file, all legal principles in the file, each and every cotext in the file \n \n 
    Follow this template for all the summaries like a bible:\n 
    1. Case Title

	•	Case Name: [e.g., ABC Corp. vs. XYZ Ltd.]
	•	Court: [e.g., Supreme Court of India]
	•	Date of Judgment: [e.g., 23rd August 2024]
	•	Citation: [e.g., 2024 SCC 123]

2. Background and Context

	•	Brief Overview: A concise summary of the case background, including relevant events leading up to the legal dispute.
	•	Key Issues: A list of the main legal questions or issues presented in the case.

3. Legal Principles Involved

	•	Relevant Statutes and Provisions: List of statutory provisions, rules, and regulations relevant to the case.
	•	Precedents Cited: Key past judgments cited during the case.
	•	Legal Doctrines: Any specific legal doctrines or principles applied in the judgment.

4. Arguments Presented

	•	Plaintiff’s Argument: Summary of the arguments and claims made by the plaintiff.
	•	Defendant’s Argument: Summary of the arguments and defenses presented by the defendant.

5. Court’s Analysis and Reasoning

	•	Key Findings: Important observations and findings made by the court.
	•	Interpretation of Law: How the court interpreted the relevant legal provisions and precedents.
	•	Application of Law: How the court applied the law to the facts of the case.

6. Judgment

	•	Final Decision: The outcome of the case (e.g., in favor of the plaintiff/defendant).
	•	Relief Granted: Any relief or damages awarded, if applicable.
	•	Orders: Specific orders or directives issued by the court.

7. Implications

	•	Impact on Law: How the judgment impacts existing law or legal practice.
	•	Future Relevance: The potential influence of the case on future legal decisions or cases.
	•	Broader Context: Any broader implications, such as social, economic, or political impact.

8. Summary Points

	•	Key Takeaways: Bullet points summarizing the most critical aspects of the case, suitable for quick reference.

9. References

	•	Citations: Full citations of any statutes, cases, or legal texts referenced in the summary.
	•	Further Reading: Suggestions for further reading or related cases.
    '''
)

In [ ]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: Gemini
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)

In [ ]:
query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(),
    llm=Settings.llm,
    qa_prompt=qa_prompt,
)

In [ ]:
response = query_engine.query("What is the case about?")

In [ ]:
print(response)

In [1]:
from agent import RAGPipeline

rag_pipeline = RAGPipeline()

rag_pipeline.load_documents(input_dir='summaries', input_files=['cases/case4.txt'])
rag_pipeline.split_documents()
rag_pipeline.create_indices()
rag_pipeline.create_query_engines()
rag_pipeline.create_tools()
rag_pipeline.create_router_engine()

/Users/lakshya/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lakshya/miniforge3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/lakshya/miniforge3/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/lakshya/mi

In [2]:
response = rag_pipeline.query("A joint venture between two Indian companies and a foreign company entered into a contract for the construction of a power plant. The contract provided that the arbitration would be held in London, but the joint venture company filed an application for interim relief under Section 9 of the Arbitration and Conciliation Act, 1996 before the Bombay High Court. The foreign company argues that the Bombay High Court lacks jurisdiction as the seat of arbitration was agreed to be London. Examine the legal principles and jurisdictional considerations in this case.")

Selecting query engine 1: The question asks for an examination of legal principles and jurisdictional considerations, specifically referencing Section 9 of the Arbitration and Conciliation Act, 1996. This indicates a need for retrieving specific legal information related to the query..


In [3]:
response

'The Bombay High Court\'s jurisdiction in this case would be determined by the interplay of the contract\'s arbitration clause, the parties\' conduct, and the provisions of the Arbitration and Conciliation Act, 1996 (Act, 1996).\n\n**Legal Principles and Jurisdictional Considerations:**\n\n* **Seat of Arbitration:** The contract\'s stipulation that arbitration be held in London is crucial. The "seat" of arbitration determines the applicable law and the courts with supervisory jurisdiction over the arbitration proceedings.  \n* **Party Autonomy:** Parties are generally free to agree on the terms of their contract, including the place of arbitration. This principle of party autonomy is enshrined in the Act, 1996.\n* **Section 9 of the Act, 1996:** This section allows parties to an arbitration agreement to apply to the court for interim measures before or during the arbitral proceedings. However, the court\'s jurisdiction to grant interim relief is not unlimited and must be exercised with